In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import shutil
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, Callback, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.regularizers import l2

from sklearn.utils.class_weight import compute_class_weight
from shutil import copyfile
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix


In [ ]:
# Path data
train_dir = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Training_dataset\bahan_training\train"
val_dir = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Training_dataset\bahan_training\val"
test_dir = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Training_dataset\dataset_testing"

## Data inspection

In [ ]:
# Periksa label data
def display_sample_images(class_dir, class_names, sample_size=5):
    for class_name in class_names:
        print(f"Class: {class_name}")
        path = os.path.join(class_dir, class_name)
        images = random.sample(os.listdir(path), sample_size)
        plt.figure(figsize=(15, 3))
        for i, image in enumerate(images):
            plt.subplot(1, sample_size, i + 1)
            img = Image.open(os.path.join(path, image))
            plt.imshow(img)
            plt.title(class_name)
            plt.axis('off')
        plt.show()

class_names = ['Depression', 'Healthy']
display_sample_images(train_dir, class_names)

In [ ]:
# Karena dataset tidak balance jadi yang healthy 253 image
def reduce_class_files(src_directory, num_files_to_remove):
    files = [file for file in os.listdir(src_directory) if os.path.isfile(os.path.join(src_directory, file))]
    
    if num_files_to_remove > len(files):
        raise ValueError("Number of files to remove exceeds the number of available files")

    files_to_remove = random.sample(files, num_files_to_remove)
    
    for file in files_to_remove:
        os.remove(os.path.join(src_directory, file))
        print(f"Removed {file}")

Healthy_dir = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\dataset_balanced\Healthy"
reduce_class_files(Healthy_dir, 1168)

# # Example usage for validation dataset
# val_depression_dir = r'E:\Vscode\Tugas Anime\Workspace\Bahan_Training_V4\val\Healthy'
# # Assume you want to remove 43 images from the validation dataset
# reduce_class_files(val_depression_dir, 43)

In [ ]:
# fungsi ngesplit
def copy_random_files_unique(source_folder: str, dest_folders: dict):
    # List all files in the source folder
    all_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    
    total_files_needed = sum(dest_folders.values())
    if len(all_files) < total_files_needed:
        raise ValueError("Not enough files in the source folder to fill all destination folders uniquely.")
    
    # Shuffle files to randomize selection
    random.shuffle(all_files)

    used_files = set()

    # Iterate over destination folders and their required file counts
    for dest_folder, num_files in dest_folders.items():
        # Create the folder if it doesn't exist
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        selected_files = 0
        file_index = 0
        
        # Select and copy files to the current destination folder
        while selected_files < num_files:
            file_name = all_files[file_index]
            if file_name not in used_files:
                shutil.copy(os.path.join(source_folder, file_name), dest_folder)
                used_files.add(file_name)
                selected_files += 1
            file_index += 1  # Move to the next file
            
            # This shouldn't happen with earlier checks but adds robustness
            if file_index >= len(all_files):
                raise RuntimeError("Ran out of unique files to select from.")

# Example usage
source_folder = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\dataset_balanced\Depression"
dest_folders = {
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\bahan_training\train\Depression": 1139,
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\bahan_training\val\Depression": 162,
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\dataset_testing\Depression": 325
}

copy_random_files_unique(source_folder, dest_folders)


In [ ]:
# fungsi ngesplit
def copy_random_files_unique(source_folder: str, dest_folders: dict):
    # List all files in the source folder
    all_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    
    total_files_needed = sum(dest_folders.values())
    if len(all_files) < total_files_needed:
        raise ValueError("Not enough files in the source folder to fill all destination folders uniquely.")
    
    # Shuffle files to randomize selection
    random.shuffle(all_files)

    used_files = set()

    # Iterate over destination folders and their required file counts
    for dest_folder, num_files in dest_folders.items():
        # Create the folder if it doesn't exist
        if not os.path.exists(dest_folder):
            os.makedirs(dest_folder)
        
        selected_files = 0
        file_index = 0
        
        # Select and copy files to the current destination folder
        while selected_files < num_files:
            file_name = all_files[file_index]
            if file_name not in used_files:
                shutil.copy(os.path.join(source_folder, file_name), dest_folder)
                used_files.add(file_name)
                selected_files += 1
            file_index += 1  # Move to the next file
            
            # This shouldn't happen with earlier checks but adds robustness
            if file_index >= len(all_files):
                raise RuntimeError("Ran out of unique files to select from.")

# Example usage
source_folder = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\dataset_balanced\Healthy"
dest_folders = {
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\bahan_training\train\Healthy": 1139,
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\bahan_training\val\Healthy": 162,
    r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_5\Training_dataset\dataset_testing\Healthy": 325
}

copy_random_files_unique(source_folder, dest_folders)


In [ ]:
# Periksa ketidakseimbangan data
def count_samples_per_class(class_dir, class_names):
    counts = {}
    for class_name in class_names:
        path = os.path.join(class_dir, class_name)
        counts[class_name] = len(os.listdir(path))
    return counts

# Check training data balance
train_counts = count_samples_per_class(train_dir, class_names)
print("Training Data:", train_counts)

# Check validation data balance
val_counts = count_samples_per_class(val_dir, class_names)
print("Validation Data:", val_counts)


### Version 1

In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# validation_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary'
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    # shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = ['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    shuffle=False
)

# pembuatan model
def create_model():
    model = Sequential([
        # Layer 1
        Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(16, 24, 3)),
        # Dropout(0.2),
        Dropout(0.1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Layer 2
        Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
        # Dropout(0.5),
        Dropout(0.1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Flatten
        Flatten(),
        # Fully connected layer
        Dense(32, activation='relu'),
        # Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
        # Dropout(0.5),
        Dropout(0.1),
        # Dense(1, activation='sigmoid')
        Dense(2, activation='softmax')
    ])
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer,
                # loss='binary_crossentropy',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'AUC'])
    return model

model = create_model()
model.summary()

# # Save the model architecture as an image file
# plot_model(model, to_file=r"E:\Vscode\Tugas Anime\Workspace\model_architecture1.png", show_shapes=True, show_layer_names=True)

# latihan
history = model.fit(train_generator,
                    # steps_per_epoch=len(train_generator),
                    epochs=30,
                    verbose=1,
                    validation_data = validation_generator,
                    # validation_steps=len(validation_generator)
                    )

# Evaluating the model on the test set
test_loss, test_acc, test_auc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss}")
print(f"Test AUC: {test_auc:.2f}")

# Predicting on the entire test set
predictions = model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
class_labels = list(test_generator.class_indices.keys())

# Calculate sensitivity and specificity
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
model.save('ridho_exe5_model1.h5')

### Version 2

In [ ]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# validation_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)


train_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary'
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    # shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = ['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    shuffle=False
)

# pembuatan model
def create_model():
    model = Sequential([
        # Layer 1
        Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(16, 24, 3)),
        # Dropout(0.2),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Layer 2
        Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
        # Dropout(0.5),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Flatten
        Flatten(),
        # Fully connected layer
        Dense(32, activation='relu'),
        # Dropout(0.5),
        # Dense(1, activation='sigmoid')
        Dense(2, activation='softmax')
    ])
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer,
                # loss='binary_crossentropy',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'AUC'])
    return model

model = create_model()
model.summary()

# latihan
history = model.fit(train_generator,
                    # steps_per_epoch=len(train_generator),
                    epochs=30,
                    verbose=1,
                    validation_data = validation_generator,
                    # validation_steps=len(validation_generator)
                    )

# Evaluating the model on the test set
test_loss, test_acc, test_auc = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss}")
print(f"Test AUC: {test_auc:.2f}")

# Predicting on the entire test set
predictions = model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
class_labels = list(test_generator.class_indices.keys())

# Calculate sensitivity and specificity
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
model.save('ridho_exe2_model2.h5')

## Version 3

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# train_datagen = ImageDataGenerator()
# validation_datagen = ImageDataGenerator()
# test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary'
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    classes=['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    # shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    classes = ['Depression', 'Healthy'],
    target_size=(16, 24),
    batch_size=4,
    # class_mode='binary',
    class_mode='categorical',
    shuffle=False
)

# pembuatan model
def create_model():
    model = Sequential([
        # Layer 1
        Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(16, 24, 3)),
        # Dropout(0.2),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Layer 2
        Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
        # Dropout(0.5),
        MaxPooling2D((2, 2), strides=(2, 2)),
        # Flatten
        Flatten(),
        # Fully connected layer
        Dense(32, activation='relu'),
        # Dropout(0.5),
        # Dense(1, activation='sigmoid')
        Dense(2, activation='softmax')
    ])
    adam_optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=adam_optimizer,
                # loss='binary_crossentropy',
                  loss='categorical_crossentropy',
                  metrics=['accuracy', 'AUC'])
    return model

model = create_model()
model.summary()

# latihan
history = model.fit(train_generator,
                    # steps_per_epoch=len(train_generator),
                    epochs=30,
                    verbose=1,
                    validation_data = validation_generator,
                    # validation_steps=len(validation_generator)
                    )

# Evaluating the model on the test set
# test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
test_loss, test_acc, test_auc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"Test Loss: {test_loss}")
print(f"Test AUC: {test_auc:.2f}")

# Predicting on the entire test set
# predictions = model.predict(test_generator, steps=len(test_generator))
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
class_labels = list(test_generator.class_indices.keys())

# Calculate sensitivity and specificity
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
model.save('ridho_exe5_model3.h5')

### Testing

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# Load your pre-trained model
model_path = r"E:\Vscode\Tugas Anime\Workspace\ridho_exe2_model2.h5"
model = load_model(model_path)

# Directory containing images for prediction
healthy_dir = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Training_dataset\dataset_testing\Healthy"

# Dictionary to store the count of predicted classes
class_count = {'Depression': 0, 'Healthy': 0}

# Dictionary to store filenames of predicted classes
predicted_files = {'Depression': [], 'Healthy': []}


# Function to process the directory
def process_directory(directory):
    for file in os.listdir(directory):
        # Ensure the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, file)
            img = load_img(img_path, target_size=(16, 24))
            img_array = img_to_array(img) /255.0
            img_array = np.expand_dims(img_array, axis=0)  # Shape (1, 16, 24, 3)

            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_labels[predicted_class_index[0]]
            
            # Increment count of the predicted class
            class_count[predicted_class] += 1
            predicted_files[predicted_class].append(file)

            # Optionally print out the file and its predicted class
            print(f"{file}: Predicted as {predicted_class}")



# Class labels mapping
class_labels = {0: 'Depression', 1: 'Healthy'}

# Process the directory
process_directory(healthy_dir)

# Output the counts
print("Classification counts:", class_count)

# Output the filenames of each predicted class
print("Files predicted as Depression:", predicted_files['Depression'])
print("Files predicted as Healthy:", predicted_files['Healthy'])


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

# Load your pre-trained model
model_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Training_dataset\hasil_training\model_3\ridho_exe4_model3.h5"
model = load_model(model_path)

# Directory containing images for prediction
depression_dir = r"Eksperimen_4\Training_dataset\dataset_testing\Depression"

# Dictionary to store the count of predicted classes
class_count = {'Depression': 0, 'Healthy': 0}

# Dictionary to store filenames of predicted classes
predicted_files = {'Depression': [], 'Healthy': []}

# Function to process the directory
def process_directory(directory):
    for file in os.listdir(directory):
        # Ensure the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, file)
            img = load_img(img_path, target_size=(16, 24))
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)  # Shape (1, 16, 24, 3)

            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions, axis=1)
            predicted_class = class_labels[predicted_class_index[0]]
            
            # Increment count of the predicted class
            class_count[predicted_class] += 1
            predicted_files[predicted_class].append(file)

            # Optionally print out the file and its predicted class
            print(f"{file}: Predicted as {predicted_class}")

# Class labels mapping
class_labels = {0: 'Depression', 1: 'Healthy'}

# Process the directory
process_directory(depression_dir)

# Output the counts
print("Classification counts:", class_count)

# Output the filenames of each predicted class
print("Files predicted as Depression:", predicted_files['Depression'])
print("Files predicted as Healthy:", predicted_files['Healthy'])
